<a href="https://colab.research.google.com/github/krjohnn/LU-DM-26m/blob/main/MD1/NOT33doubleNP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import plotly.graph_objects as go

# Make 2 data points
x = torch.tensor([[0.0,1.0]], dtype=torch.double)
y = torch.tensor([1.0,0.0], dtype=torch.double)

w_all = [w for w in range(-15, 10, 1)]
b_all = [b for b in range(-10, 10, 1)]

loss_landscape = []
for b in b_all:
        tmp = []
        for W in w_all:
            b = torch.tensor([b], dtype=torch.double)
            W = torch.tensor([[W]], dtype=torch.double)
            y_pred = torch.sigmoid(W.mm(x) + b)
            loss = (y_pred - y).pow(2).sum()/2
            tmp.append(loss.item())
        loss_landscape.append(tmp)

fig = go.Figure(data=[go.Surface(x=w_all, y=b_all, z=loss_landscape)])

fig.update_layout(scene=dict(xaxis_title='W', yaxis_title='b',zaxis_title='Loss'))

fig.show()


In [ ]:
x_plot = []
y_plot = []
z_plot = []

    # Construct a linear model
b = torch.tensor([-1.0], dtype=torch.double, requires_grad=True)
W = torch.tensor([[4.0]], dtype=torch.double, requires_grad=True)

optimizer = torch.optim.SGD([b,W], lr=50.0)

for t in range(12):
    y_pred = torch.sigmoid(W.mm(x) + b)
    loss = (y_pred - y).pow(2).sum()/2
    print(t, W.item(), b.item(), loss.item())

    x_plot.append(W.item())
    y_plot.append(b.item())
    z_plot.append(loss.item() + 0.03)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# Nosakām punktu skaitu
num_points = len(x_plot)

# 1. Definējam krāsas: viss melns, sākums sarkans, beigas zilas
point_colors = ['black'] * num_points
point_colors[0] = 'red'    # Sākums
point_colors[-1] = 'blue'  # Beigas

# 2. (Pēc izvēles) Definējam izmērus: sākums un beigas lielāki
point_sizes = [8] * num_points  # Vidus punkti mazāki (izmērs 6)
point_sizes[0] = 10             # Sākums liels (izmērs 10)
point_sizes[-1] = 10            # Beigas lielas (izmērs 10)

# Pievienojam trajektoriju ar līnijām UN marķieriem
fig.add_trace(go.Scatter3d(
    x=x_plot, y=y_plot, z=z_plot,
    mode="lines+markers",         # SVARĪGI: zīmējam gan līniju, gan punktus
    line=dict(color="lime", width=4), # Līnija nedaudz šaurāka, lai izceltu punktus
    marker=dict(
        color=point_colors,       # Mūsu definēto krāsu saraksts
        size=point_sizes,         # Mūsu definēto izmēru saraksts
        symbol='circle',
        line=dict(color='white', width=1) # Balta kontūra punktiem smukumam
    ),
    name="Gradienta kritums"
))

fig.show()

# Rezutlātu pābaude

In [ ]:
with torch.no_grad():
    y_pred_0 = torch.sigmoid(W * 0.0 + b).item()
    y_pred_1 = torch.sigmoid(W * 1.0 + b).item()

print(f"Beigu svari: W = {W.item():.4f}, b = {b.item():.4f}")
print(f"Ja x=0, prognoze ir {y_pred_0:.4f} (Vajadzētu būt tuvu 1)")
print(f"Ja x=1, prognoze ir {y_pred_1:.4f} (Vajadzētu būt tuvu 0)")